In [1]:
from utils.dataset import AirDelayDataset
import utils.airdelay_plot as plot

In [2]:
data = AirDelayDataset('../datasets/app-dataset/full/train.csv',
                       airport_data_path='../datasets/AIRPORTS_INFO.csv')

Initializing Spark...
Spark stand by (Master=local, AppName=1e4a7d4b-8663-4833-9b9e-d3a26989b98c)
Load datasets from ../datasets/app-dataset/full/train.csv
                                                     0
Spark Master                                     local
Spark APP Name    1e4a7d4b-8663-4833-9b9e-d3a26989b98c
Data Path       ../datasets/app-dataset/full/train.csv
Data Count                                      121513


In [3]:
d = data.get_date_period('2009-01-01', '2009-01-02')
fig = plot.map.heatmap(d, 'origin', 'royalblue', 'Origin Port')
fig = plot.map.heatmap(d, 'dest', 'orange', 'Destination Port', fig=fig)
fig.show()

In [4]:
d = data.get_date_period('2009-01-01', '2009-01-02')
fig = plot.map.heatmap(d, 'origin', 'royalblue', 'Origin Port')
fig = plot.map.heatmap(d.get_delay(), 'origin', 'red', 'Delay', fig=fig)
fig.show()

In [5]:
d = data.get_date_period('2009-01-01', '2009-01-02')
fig = plot.map.heatmap(d, 'dest', 'royalblue', 'Destination Port')
fig = plot.map.heatmap(d.get_delay(), 'dest', 'red', 'Delay', fig=fig)
fig.show()

In [6]:
fig = plot.map.heatmap(data.get_delay(), 'dest')
fig.show()

In [7]:
d = data.get_date_period('2009-01-01', '2009-01-02')
fig = plot.map.linemap(d, name='flight', color='royalblue')
fig = plot.map.linemap(d.get_delay(), color='red', name='delay', fig=fig)
fig.show()

In [8]:
data.createTempView().show(50)


+-------------------+---------+-------+
|           col_name|data_type|comment|
+-------------------+---------+-------+
|                _c0|      int|   null|
|            FL_DATE|     date|   null|
|         OP_CARRIER|   string|   null|
|  OP_CARRIER_FL_NUM|      int|   null|
|             ORIGIN|   string|   null|
|               DEST|   string|   null|
|       CRS_DEP_TIME|   double|   null|
|           DEP_TIME|   double|   null|
|          DEP_DELAY|   double|   null|
|           TAXI_OUT|   double|   null|
|         WHEELS_OFF|   double|   null|
|          WHEELS_ON|   double|   null|
|            TAXI_IN|   double|   null|
|       CRS_ARR_TIME|   double|   null|
|           ARR_TIME|   double|   null|
|          ARR_DELAY|   double|   null|
|          CANCELLED|   double|   null|
|  CANCELLATION_CODE|   string|   null|
|           DIVERTED|   double|   null|
|   CRS_ELAPSED_TIME|   double|   null|
|ACTUAL_ELAPSED_TIME|   double|   null|
|           AIR_TIME|   double|   null|


In [9]:
print("This type of delay is longer than 0 minutes")
data.sql("""SELECT
(SELECT count(WEATHER_DELAY) FROM flightdata WHERE WEATHER_DELAY>0) AS WEATHER_DELAY, 
(SELECT count(LATE_AIRCRAFT_DELAY) FROM flightdata WHERE LATE_AIRCRAFT_DELAY>0)AS LATE_AIRCRAFT_DELAY, 
(SELECT count(NAS_DELAY) FROM flightdata WHERE NAS_DELAY>0) AS NAS_DELAY, 
(SELECT count(SECURITY_DELAY) FROM flightdata WHERE SECURITY_DELAY>0) AS SECURITY_DELAY
FROM flightdata
LIMIT 1""").show()

This type of delay is longer than 0 minutes
+-------------+-------------------+---------+--------------+
|WEATHER_DELAY|LATE_AIRCRAFT_DELAY|NAS_DELAY|SECURITY_DELAY|
+-------------+-------------------+---------+--------------+
|         1303|              11860|    12587|            65|
+-------------+-------------------+---------+--------------+



In [10]:
print("Longest delay is of this type")
data.sql("""SELECT 
(CASE
    WHEN LATE_AIRCRAFT_DELAY>0 AND LATE_AIRCRAFT_DELAY=greatest(WEATHER_DELAY, LATE_AIRCRAFT_DELAY, NAS_DELAY, SECURITY_DELAY) THEN "LATE_AIRCRAFT_DELAY"
    WHEN WEATHER_DELAY>0 AND WEATHER_DELAY=greatest(WEATHER_DELAY, LATE_AIRCRAFT_DELAY, NAS_DELAY, SECURITY_DELAY) THEN "WEATHER_DELAY"
    WHEN NAS_DELAY>0 AND NAS_DELAY=greatest(WEATHER_DELAY, LATE_AIRCRAFT_DELAY, NAS_DELAY, SECURITY_DELAY) THEN "NAS_DELAY"
    WHEN SECURITY_DELAY>0 AND SECURITY_DELAY=greatest(WEATHER_DELAY, LATE_AIRCRAFT_DELAY, NAS_DELAY, SECURITY_DELAY) THEN "SECURITY_DELAY"
    ELSE "UNKNOWN"
END) AS delay_type, 
count(*) AS count
FROM flightdata
WHERE WEATHER_DELAY>0 OR LATE_AIRCRAFT_DELAY>0 OR NAS_DELAY>0 OR SECURITY_DELAY>0
GROUP BY delay_type""").show()

Longest delay is of this type
+-------------------+-----+
|         delay_type|count|
+-------------------+-----+
|      WEATHER_DELAY|  708|
|LATE_AIRCRAFT_DELAY|10338|
|          NAS_DELAY| 8696|
|     SECURITY_DELAY|   42|
+-------------------+-----+



In [11]:
data.sql("CREATE OR REPLACE TEMPORARY VIEW flightbycarrier AS SELECT OP_CARRIER, count(*) AS TOTAL_FLIGHTS FROM flightdata GROUP BY OP_CARRIER ORDER BY OP_CARRIER ASC")
data.sql("CREATE OR REPLACE TEMPORARY VIEW latearrivalbycarrier AS SELECT OP_CARRIER, count(*) AS LATE_ARRIVALS FROM flightdata WHERE ARR_DELAY>0 GROUP BY OP_CARRIER ORDER BY OP_CARRIER ASC")

print("Late arrivals by carrier")
data.sql("""
SELECT f.OP_CARRIER, TOTAL_FLIGHTS, LATE_ARRIVALS, (LATE_ARRIVALS/TOTAL_FLIGHTS) AS laterate
FROM flightbycarrier f, latearrivalbycarrier l
WHERE f.OP_CARRIER = l.OP_CARRIER
ORDER BY OP_CARRIER ASC
""").show()

Late arrivals by carrier
+----------+-------------+-------------+-------------------+
|OP_CARRIER|TOTAL_FLIGHTS|LATE_ARRIVALS|           laterate|
+----------+-------------+-------------+-------------------+
|        9E|         2094|          718| 0.3428844317096466|
|        AA|        13254|         5042|0.38041346008752075|
|        AS|         3298|         1098| 0.3329290479078229|
|        B6|         4973|         1997| 0.4015684697365775|
|        CO|         1463|          644|0.44019138755980863|
|        DL|        15615|         4964|0.31789945565161704|
|        EV|         9124|         3587|0.39313897413415166|
|        F9|         1795|          807|0.44958217270194983|
|        FL|         2440|          888| 0.3639344262295082|
|        G4|          190|           78| 0.4105263157894737|
|        HA|         1452|          508|  0.349862258953168|
|        MQ|         6133|         2475|0.40355454100766347|
|        NK|         1138|          417| 0.3664323374340949|

In [12]:
data.sql("CREATE OR REPLACE TEMPORARY VIEW flightbyflightnumber AS SELECT CONCAT(OP_CARRIER, OP_CARRIER_FL_NUM) AS FLIGHT_NUMBER, count(*) AS TOTAL_FLIGHTS FROM flightdata GROUP BY FLIGHT_NUMBER ORDER BY FLIGHT_NUMBER ASC")
data.sql("CREATE OR REPLACE TEMPORARY VIEW latearrivalbyflightnumber AS SELECT CONCAT(OP_CARRIER, OP_CARRIER_FL_NUM) AS FLIGHT_NUMBER, count(*) AS LATE_ARRIVALS FROM flightdata WHERE ARR_DELAY>0 GROUP BY FLIGHT_NUMBER ORDER BY FLIGHT_NUMBER ASC")

print("Late arrivals by flight number")
data.sql("""
SELECT f.FLIGHT_NUMBER, TOTAL_FLIGHTS, LATE_ARRIVALS, (LATE_ARRIVALS/TOTAL_FLIGHTS) AS laterate
FROM flightbyflightnumber f, latearrivalbyflightnumber l
WHERE f.FLIGHT_NUMBER = l.FLIGHT_NUMBER
ORDER BY FLIGHT_NUMBER ASC
""").show()

Late arrivals by flight number
+-------------+-------------+-------------+------------------+
|FLIGHT_NUMBER|TOTAL_FLIGHTS|LATE_ARRIVALS|          laterate|
+-------------+-------------+-------------+------------------+
|       9E2114|            1|            1|               1.0|
|       9E2116|            1|            1|               1.0|
|       9E2130|            2|            1|               0.5|
|       9E2137|            1|            1|               1.0|
|       9E2150|            1|            1|               1.0|
|       9E2151|            1|            1|               1.0|
|       9E2179|            1|            1|               1.0|
|       9E2185|            1|            1|               1.0|
|       9E2189|            2|            1|               0.5|
|       9E2191|            1|            1|               1.0|
|       9E2204|            2|            1|               0.5|
|       9E2225|            1|            1|               1.0|
|       9E2240|         

In [13]:
data.sql("CREATE OR REPLACE TEMPORARY VIEW flightbyroute AS SELECT ORIGIN, DEST, count(*) AS TOTAL_FLIGHTS FROM flightdata GROUP BY ORIGIN, DEST ORDER BY ORIGIN, DEST ASC")
data.sql("CREATE OR REPLACE TEMPORARY VIEW latearrivalbyroute AS SELECT ORIGIN, DEST, count(*) AS LATE_ARRIVALS FROM flightdata WHERE ARR_DELAY>0 GROUP BY ORIGIN, DEST ORDER BY ORIGIN, DEST ASC")

print("Late arrivals by route")
data.sql("""
SELECT f.ORIGIN, f.DEST, TOTAL_FLIGHTS, LATE_ARRIVALS, (LATE_ARRIVALS/TOTAL_FLIGHTS) AS laterate
FROM flightbyroute f, latearrivalbyroute l
WHERE f.ORIGIN=l.ORIGIN
AND f.DEST=l.DEST
ORDER BY ORIGIN, DEST ASC
""").show()

Late arrivals by route
+------+----+-------------+-------------+-------------------+
|ORIGIN|DEST|TOTAL_FLIGHTS|LATE_ARRIVALS|           laterate|
+------+----+-------------+-------------+-------------------+
|   ABE| ATL|           16|            7|             0.4375|
|   ABE| DTW|           20|            6|                0.3|
|   ABE| MCO|            3|            2| 0.6666666666666666|
|   ABE| MYR|            1|            1|                1.0|
|   ABE| ORD|           13|            5|0.38461538461538464|
|   ABI| DFW|           41|           14|0.34146341463414637|
|   ABQ| ATL|           17|            9| 0.5294117647058824|
|   ABQ| BWI|            9|            2| 0.2222222222222222|
|   ABQ| DAL|           48|           20| 0.4166666666666667|
|   ABQ| DEN|           47|           16| 0.3404255319148936|
|   ABQ| DFW|           45|           14| 0.3111111111111111|
|   ABQ| ELP|            3|            2| 0.6666666666666666|
|   ABQ| HOU|           18|           10| 0.555

In [17]:
print("Number of carriers for each flight route")
data.sql("""
SELECT f.ORIGIN, f.DEST, count(DISTINCT OP_CARRIER) AS number_of_carriers
FROM flightdata f
GROUP BY ORIGIN, DEST
ORDER BY ORIGIN, DEST ASC
""").show()


Number of carriers for each flight route
+------+----+------------------+
|ORIGIN|DEST|number_of_carriers|
+------+----+------------------+
|   ABE| ATL|                 3|
|   ABE| CLT|                 3|
|   ABE| DTW|                 4|
|   ABE| FLL|                 1|
|   ABE| IAD|                 1|
|   ABE| MCO|                 1|
|   ABE| MYR|                 1|
|   ABE| ORD|                 4|
|   ABE| SFB|                 1|
|   ABI| DFW|                 2|
|   ABQ| AMA|                 1|
|   ABQ| ATL|                 1|
|   ABQ| BWI|                 1|
|   ABQ| DAL|                 1|
|   ABQ| DEN|                 7|
|   ABQ| DFW|                 2|
|   ABQ| ELP|                 1|
|   ABQ| HOU|                 1|
|   ABQ| IAD|                 1|
|   ABQ| IAH|                 6|
+------+----+------------------+
only showing top 20 rows

